In [18]:
import requests as r
import datetime as dt
import csv
from PIL import Image
from IPython.display import display

In [2]:
url = "https://api.covid19api.com/dayone/country/brazil"
resp = r.get(url)
print(resp.status_code)

200


In [ ]:
raw_data = resp.json()  # converte os dados recebidos no formtado json
raw_data[0]

In [4]:
final_data = []

for obs in raw_data:
    final_data.append([obs['Confirmed'], obs['Deaths'], obs['Recovered'], obs['Active'], obs['Date']])

final_data.insert(0, ['confirmados', 'obitos', 'recuperados', 'ativos', 'data'])

In [ ]:
final_data

In [6]:
CONFIRMADOS = 0
OBITOS = 1
RECUPERADOS = 2
ATIVOS = 3
DATA = 4

In [7]:
# para mudar os dados no campo DATA, eu preciso pegar cada uma das posições do campo e fazer a alteração
# por isso que nós referenciamos com a posição i, na posicao DATA, e fazemos o slice
for i in range(1, len(final_data)):
    final_data[i][DATA] = final_data[i][DATA][:10]

In [8]:
# o calculo da diferença entre duas datas retorna um tipo timedelta, do qual podemos retirar os tempos que queremos

In [9]:
with open('brasil-covid.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(final_data)

In [10]:
for i in range(1, len(final_data)):
    final_data[i][DATA] = dt.datetime.strptime(final_data[i][DATA], '%Y-%m-%d')

In [ ]:
final_data

In [19]:
# REPOSITÓRIO DE FUNCOES

# funcao que cria os labels
def get_datasets(y, labels):
    if type(y[0] == list):
        datasets = []  # essa lista vai conter os valores de y e label respectivo de cada um dela

        for i in range(len(y)):
            datasets.append({
                'label': labels[i],
                'data': y[i]
            })
        
        return datasets
    else:
        return [{
            'label': labels[0],
            'data': y
        }]

# funcao para definir o titulo do gráfico
def set_title(title=''):
    if title != '':
        display = 'true'
    else:
        display = 'false'
    return {
        'title': title,
        'display': display
    }

# funcao que cria o dicionario que representa o gráfico
def create_chart(x, y, labels, kind='bar', title=''):

    datasets = get_datasets(y, labels)
    options = set_title(title)

    chart = {
        'type': kind,
        'data': {
            'labels': x,
            'datasets': datasets
        },
        'options': options
    }

    return chart

# retorna o valor binario da imagem
def get_api_chart(chart):
    url_base = 'https://quickchart.io/chart'
    resp = r.get(f'{url_base}?c={str(chart)}')  # essa chamada vai retornar o arquivo de imagem
    return resp.content  # content retorna o valor binários


# funcao para salvar a imagem
def save_image(path, content):
    with open(path, 'wb') as image:    # w significa que é modo de escrita, e b significa que está trabalhando com um arquivo binário
        image.write(content)


# funcao para importar a imagem
def display_image(path):
    img_pil = Image(path)
    display(img_pil)

In [ ]:
y_data_1 = []
for obs in final_data[1::10]:  # vai pular o header, e fazer steps de 10 em 10 registros
    y_data_1.append(obs[CONFIRMADOS])

y_data_2 = []
for obs in final_data[1::10]:  # vai pular o header, e fazer steps de 10 em 10 registros
    y_data_2.append(obs[RECUPERADOS])

labels = ['Confirmados', 'Recuperados']

x = []
for obs in final_data[1::10]:
    x.append(obs[DATA])